In [1]:
import networkx as nx
import random
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import pandas as pd

In [2]:
def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5


def plot_graph_v2(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

    # Lấy thuộc tính xác suất và trọng số của các cạnh
    edge_probabilities = nx.get_edge_attributes(G, 'probability')
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Kết hợp xác suất và trọng số vào nhãn cạnh
    edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=200, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Weighted Directed Graph Visualization')
    plt.show()
    
def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0



def calculate_f_beta_v3(G_S, beta, S, T):
    num_edges = G_S.number_of_edges()    
    if len(S) > 0 and len(T) > 0:
        f_beta = (weighted_average_edge_probability(G_S) - beta) * (num_edges / (len(S) * len(T))**0.5)
    else:
        f_beta = 0
    return f_beta
    

def weighted_average_edge_probability(G):
    total_weighted_probability = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())  
    num_edges = G.number_of_edges()
    return total_weighted_probability / num_edges if num_edges > 0 else 0

def surplus_degree_out(G, v, beta):
    return sum(G[v][w]['weight'] *G[v][w]['probability'] - beta for w in G.successors(v) if w in G)

def surplus_degree_in(G, v, beta):
    return sum(G[u][v]['weight'] *G[u][v]['probability'] - beta for u in G.predecessors(v) if u in G)


def initialize_priority_queue_out(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_out(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue

def initialize_priority_queue_in(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_in(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue


def calculate_edge_density_v3(G):
    """Tính mật độ cạnh kỳ vọng của đồ thị G. τ"""
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_weighted_probabilities = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())
    return sum_weighted_probabilities / num_possible_edges

def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5

def trong_so_trung_binh_canh(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'weight').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

def greedy_average_surplus_degree(G, beta, S, T, c):
    H = G.copy()
    S_temp = S
    T_temp = T
    best_subgraph = H.copy()
    best_f_beta = calculate_f_beta_v3(best_subgraph, beta, S_temp, T_temp)

    priority_queue_out = initialize_priority_queue_out(H, beta)
    priority_queue_in = initialize_priority_queue_in(H, beta)

    with tqdm(total=H.number_of_nodes() - 2, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            while priority_queue_out:
                _, i_min = heapq.heappop(priority_queue_out)
                if i_min in H:
                    break

            while priority_queue_in:
                _, j_min = heapq.heappop(priority_queue_in)
                if j_min in H:
                    break
            d_S = surplus_degree_out(H, i_min, beta) if i_min in H else float('inf')
            d_T = surplus_degree_in(H, j_min, beta) if j_min in H else float('inf')
            
                
        
            
            if c * d_S <= d_T / c:
                S_temp.remove(i_min)
                H.remove_node(i_min)
            else:
                T_temp.remove(j_min)
                H.remove_node(j_min)

            current_f_beta = calculate_f_beta_v3(H, beta, S_temp, T_temp)
            
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}
            
            if current_f_beta > best_f_beta:
                best_f_beta = current_f_beta
                best_subgraph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

            # # Cập nhật lại hàng đợi ưu tiên
            # priority_queue_out = initialize_priority_queue_out(H, beta)
            # priority_queue_in = initialize_priority_queue_in(H, beta)
            
            pbar.update(1)  # Cập nhật tiến trình mỗi khi một đỉnh được loại bỏ
    
    return best_subgraph



In [3]:
def update_surplus_degrees(H, beta, removed_node, priority_queue_out, priority_queue_in):
    if removed_node in H:  # Kiểm tra nếu đỉnh còn tồn tại trong đồ thị
        for neighbor in list(H.successors(removed_node)):
            if neighbor in H:  # Kiểm tra nếu đỉnh kề vẫn còn trong đồ thị
                sd = surplus_degree_out(H, neighbor, beta)
                heapq.heappush(priority_queue_out, (sd, neighbor))
    
        for neighbor in list(H.predecessors(removed_node)):
            if neighbor in H:  # Kiểm tra nếu đỉnh kề vẫn còn trong đồ thị
                sd = surplus_degree_in(H, neighbor, beta)
                heapq.heappush(priority_queue_in, (sd, neighbor))


In [5]:
def update_surplus_degrees_after_removal(H, beta, neighbors, priority_queue_out, priority_queue_in):
    for neighbor in neighbors:
        if neighbor in H:
            sd_out = surplus_degree_out(H, neighbor, beta)
            heapq.heappush(priority_queue_out, (sd_out, neighbor))
            
            sd_in = surplus_degree_in(H, neighbor, beta)
            heapq.heappush(priority_queue_in, (sd_in, neighbor))

def greedy_average_surplus_degree(G, beta, S, T, c):
    H = G.copy()
    S_temp = S.copy()
    T_temp = T.copy()
    best_subgraph = H.copy()
    best_f_beta = calculate_f_beta_v3(best_subgraph, beta, S_temp, T_temp)

    priority_queue_out = initialize_priority_queue_out(H, beta)
    priority_queue_in = initialize_priority_queue_in(H, beta)

    with tqdm(total=H.number_of_nodes() - 2, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            while priority_queue_out:
                _, i_min = heapq.heappop(priority_queue_out)
                if i_min in H:
                    break

            while priority_queue_in:
                _, j_min = heapq.heappop(priority_queue_in)
                if j_min in H:
                    break

            d_S = surplus_degree_out(H, i_min, beta) if i_min in H else float('inf')
            d_T = surplus_degree_in(H, j_min, beta) if j_min in H else float('inf')

            if c * d_S <= d_T / c:
                neighbors = list(H.successors(i_min)) + list(H.predecessors(i_min))
                if i_min in S_temp:
                    S_temp.remove(i_min)
                H.remove_node(i_min)
                update_surplus_degrees_after_removal(H, beta, neighbors, priority_queue_out, priority_queue_in)
                # Add j_min back to the priority queue if it still exists
                if j_min in H:
                    heapq.heappush(priority_queue_in, (surplus_degree_in(H, j_min, beta), j_min))
            else:
                neighbors = list(H.successors(j_min)) + list(H.predecessors(j_min))
                if j_min in T_temp:
                    T_temp.remove(j_min)
                H.remove_node(j_min)
                update_surplus_degrees_after_removal(H, beta, neighbors, priority_queue_out, priority_queue_in)
                # Add i_min back to the priority queue if it still exists
                if i_min in H:
                    heapq.heappush(priority_queue_out, (surplus_degree_out(H, i_min, beta), i_min))

            current_f_beta = calculate_f_beta_v3(H, beta, S_temp, T_temp)
            
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}

            if current_f_beta > best_f_beta:
                best_f_beta = current_f_beta
                best_subgraph = H.copy()

            pbar.update(1)

    return best_subgraph


In [6]:
# Đọc đồ thị từ file và gán xác suất ngẫu nhiên cho các cạnh
random.seed(99)
G = nx.DiGraph()

with open('data/facebook_combined.txt', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        source, target = map(int, line.split())
        probability = random.random()  # Xác suất ngẫu nhiên giữa 0 và 1
        weight = random.random()
        # G.add_edge(source, target, probability=probability, weight=weight)
        if random.choice([True, False]):
            G.add_edge(source, target, probability=probability, weight=weight)
        else:
            G.add_edge(target, source, probability=probability, weight=weight)
#         G.add_edge(source, target, probability=probability, weight=weight)
        

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)

# print("Set S:", S)
# print("Set T:", T)


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
std_weight = np.std(weights)

    
print("Giá trị cạnh trung bình của đồ thị :", mean_probability)
print("Độ lệch chuẩn xác xuất:", std_deviation)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ cạnh kì vọng của đồ thị con :", calculate_edge_density_v3(G))
print("Độ lệch chuẩn trọng số :",std_weight)
print("Trung bình trọng số xác xuất :",weighted_average_edge_probability(G))





Number of nodes: 4039
Number of edges: 88234
Giá trị cạnh trung bình của đồ thị : 0.49955353391381374
Độ lệch chuẩn xác xuất: 0.2885560933098177
Trọng số cạnh trung bình của đồ thị là: 0.4992135267039643
Mật độ cạnh kì vọng của đồ thị con : 0.0013468005287093304
Độ lệch chuẩn trọng số : 0.2891885145847438
Trung bình trọng số xác xuất : 0.2489473330074043


In [7]:
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)
beta = 0.6 * 0.6 
c = len(S) / len(T)
ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta, S, T, c)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
std_weight = np.std(weights)

print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn xác xuất :", std_deviation)
print("Độ lệch chuẩn trọng số:", std_weight)
print("Trung bình trọng số xác xuất :",weighted_average_edge_probability(ObsEmailEuCore01))





Processing: 4039node [03:49, 17.61node/s]                                                                              

Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là: 0.6398295834760939
Số đỉnh trong đồ thị con của thuật toán obs  là: 3
Số cạnh trong đồ thị con của thuật toán obs  là: 3
Mật độ kì cạnh vọng của thuật toán obs  là: 0.31991479173804693
Xác xuất cạnh trung bình của thuật toán obs là: 0.8514822479969192
 trọng số cạnh trung bình của thuật toán obs là: 0.7812816006765054
Độ lệch chuẩn xác xuất : 0.17013971553037005
Độ lệch chuẩn trọng số: 0.1504317096326951
Trung bình trọng số xác xuất : 0.6398295834760939
